In [2]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)
# tf.config.experimental.set_virtual_device_configuration(
#     physical_devices[0],
#     [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1250*3)])
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam

# Define paths and constants
data_dir = 'dataset/ISIC_2019_Training_Input'  # Replace with the path to your image directory
csv_file = 'ISIC_2019_Training_GroundTruth.csv'  # Replace with the path to your CSV file
save_model_path = 'LatestModel.h5'  # Replace with the desired path to save the model

# Read the CSV file
df = pd.read_csv(csv_file)
df['image'] = df['image'].apply(lambda x: x + '.jpg')
df['MEL'] = df['MEL'].astype(str)

# Separate the malignant and benign classes
df_malignant = df[df['MEL'] == '1.0']
df_benign = df[df['MEL'] == '0.0']

# Determine the number of samples you want to select from each class (e.g., 1000 from each class)
num_samples_per_class = 4000

# Sample an equal number of malignant and benign images
df_sampled = pd.concat([
    df_malignant.sample(n=num_samples_per_class, random_state=42),
    df_benign.sample(n=num_samples_per_class, random_state=42)
], ignore_index=True)

# Shuffle the sampled DataFrame
df_sampled = df_sampled.sample(frac=1, random_state=42).reset_index(drop=True)

# Create data generators for training and validation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.1,
    validation_split=0.2
)

# Load and preprocess images in batches using the generator
train_gen = datagen.flow_from_dataframe(
    dataframe=df_sampled,
    directory=data_dir,
    x_col='image',
    y_col='MEL',
    target_size=(224, 224),
    batch_size=32,
    subset='training',
    class_mode='binary'
)

val_gen = datagen.flow_from_dataframe(
    dataframe=df_sampled,
    directory=data_dir,
    x_col='image',
    y_col='MEL',
    target_size=(224, 224),
    batch_size=32,
    subset='validation',
    class_mode='binary'
)




TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [2]:
# Create the model
base_model = MobileNetV2(weights='imagenet', include_top=False)
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_gen, validation_data=val_gen, epochs=10)

# Save the model
model.save(save_model_path)



C:\Program Files\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/10
200/200 [==============================] - 125s 575ms/step - loss: 0.5460 - accuracy: 0.7127 - val_loss: 0.6473 - val_accuracy: 0.6269
Epoch 2/10
200/200 [==============================] - 116s 582ms/step - loss: 0.4500 - accuracy: 0.7806 - val_loss: 0.5830 - val_accuracy: 0.7006
Epoch 3/10
200/200 [==============================] - 122s 609ms/step - loss: 0.4127 - accuracy: 0.8002 - val_loss: 0.7497 - val_accuracy: 0.6825
Epoch 4/10
200/200 [==============================] - 128s 641ms/step - loss: 0.3603 - accuracy: 0.8344 - val_loss: 0.6032 - val_accuracy: 0.7375
Epoch 5/10
200/200 [==============================] - 131s 655ms/step - loss: 0.3127 - accuracy: 0.8594 - val_loss: 0.5725 - val_accuracy: 0.7450
Epoch 6/10
200/200 [==============================] - 139s 695ms/step - loss: 0.2871 - accuracy: 0.8730 - val_loss: 0.5233 - val_accuracy: 0.7575
Epoch 7/10
200/200 [==============================] - 138s 691ms/step - loss: 0.2474 - accuracy: 0.8950 - val_loss: 0.5167 -

In [15]:
import numpy as np
from keras.models import load_model
from PIL import Image
from keras.preprocessing import image

# Load the saved model for prediction
loaded_model = load_model('LatestModel.h5')

# Load an image for prediction (replace 'path_to_image.jpg' with the actual image path)
#img_path = 'ISIC_2019_Training_Input/ISIC_0000002.jpg'
img_path = 'sc2.jpeg'
img = Image.open(img_path)
img = img.resize((224, 224))  # Resize the image to match the model's input size

# Convert the image to a numpy array
img_array = np.array(img)
img_array = np.expand_dims(img_array, axis=0)

# Make a prediction
prediction = loaded_model.predict(img_array)
print(prediction)
# Convert the prediction to a class label
if prediction[0][0] >= 0.4:
    print("Malignant")
else:
    print("Benign")


1/1 [==============================] - 1s 685ms/step
[[0.974817]]
Malignant
